In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [13]:
characters = pd.read_csv("Characters.csv")
names = characters["name"].tolist()
URL_ADDITIONS = ["-".join(name.lower().replace("\'", "").split()) for name in names]
BASE_URL = "https://www.starwars.com/databank/"

In [55]:
def get_page(url):
    try: response = requests.get(url)
    except: print(url); return None
    return BeautifulSoup(response.text, "html.parser")

def get_descriptions(url):
    soup = get_page(url)
    if soup is None:
        return None, None

def get_properties(soup):
    stats = soup.find_all("section", class_="module stats span-full-screen content-span-full-screen secondary-theme dark")
    categories = stats[0].find_all("div", class_="category")
    properties = {}
    for category in categories:
        heading = category.find_all("div", class_="heading")[0].text
        property_names = [p.text for p in category.find_all("div", class_="property-name")]
        properties[heading] = property_names
    return properties

def get_description(soup):
    discription = soup.find_all("p", class_="desc")[0].text
    return discription

def get_history(soup):
    all_referals = []
    all_text = soup.find_all("div", class_="rich-text-output")
    if len(all_text) == 0:
        return None
    minor_text_samples = all_text[0].find_all("p")
    all_history_text = ""
    for sample in minor_text_samples:
        referals = sample.find_all("a")
        for referal in referals:
            all_referals.append(referal.text.strip())
        all_history_text += " " + sample.text
    return all_history_text, all_referals

In [56]:
full_url = BASE_URL + "admiral-ackbar"
soup = get_page(full_url)
#print(soup)
text, referals = get_history(soup)
referals


['Mon Calamari',
 'Mon Cala',
 'Prince Lee-Char',
 'Separatist',
 'Quarren',
 'Alliance',
 'Empire',
 'Emperor',
 'second Death Star',
 'Endor',
 'Lando Calrissian',
 'Wedge Antilles',
 'the New Republic',
 'Jakku',
 'Leia Organa',
 'Resistance']

In [10]:
# creating dataframe objects:
names = []
descriptions = []
referals = []
properties = []
history = []

for url_addition in URL_ADDITIONS:
    full_url = BASE_URL + url_addition

['hyper-rod',
 'quick-draw-quasar',
 'steel-claw-kane',
 'the-trickster-flash-raktor',
 '2br-ntb']